In [ ]:
"""
One-dimensional Schrödinger steady-state equation.
Square potential well.
"""
import numpy as np
from scipy.integrate import odeint
from scipy.misc import derivative
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math
from scipy.integrate import quad

global r, n, Psi, Fi, X, XX
T = open("glebov-2b.txt", "w")


# potential function
def U(x):
    if (abs(x) < L):
        return V0 * pow(x - L, 2) / 9 * math.exp(-x * x)
    else:
        return W


# function (13)
def q(e, x):
    return 2.0*(e-U(x))

#this
def operT(funF, X):
    localX = [X[0] - 2 * 1.e-6]
    for x in X:
        localX.append(x)
    localX.append(X[len(X) - 1] + 2 * 1.e-6)

    localFunF = [0]
    for val in funF:
        localFunF.append(val)
    localFunF.append(0)

    fun = interp1d(localX, localFunF, kind='cubic')
    der = []
    for x in X:
        der.append(-1/2 * derivative(fun, x, dx=1.e-6, n=2, order=5))
    return der

def system1(cond1, X):
    global eee
    Y0, Y1 = cond1[0], cond1[1]
    dY0dX = Y1
    dY1dX = - q(eee, X)*Y0
    return [dY0dX, dY1dX]


def system2(cond2, XX):
    global eee
    Z0, Z1 = cond2[0], cond2[1]
    dZ0dX = Z1
    dZ1dX = - q(eee, XX)*Z0
    return [dZ0dX, dZ1dX]


# calculation of f (eq. 18; difference of derivatives)
def f_fun(e):
    global r, n, Psi, Fi, X, XX, eee
    eee = e
    """
    Cauchy problem ("forward")
    dPsi1(x)/dx = - q(e, x)*Psi(x);
    dPsi(x)/dx = Psi1(x);
    Psi(A) = 0.0
    Psi1(A)= 1.0
    """
    cond1 = [0.0, 1.0]
    sol1 = odeint(system1, cond1, X)
    Psi, Psi1 = sol1[:, 0], sol1[:, 1]
    """
    Cauchy problem ("backwards")
    dPsi1(x)/dx = - q(e, x)*Psi(x);
    dPsi(x)/dx = Psi1(x);
    Psi(B) = 0.0
    Psi1(B)= 1.0
    """
    cond2 = [0.0, 1.0]
    sol2 = odeint(system2, cond2, XX)
    Fi, Fi1 = sol2[:, 0], sol2[:, 1]
    # search of maximum value of Psi
    p1 = np.abs(Psi).max()
    p2 = np.abs(Psi).min()
    big = p1 if p1 > p2 else p2
    # scaling of Psi
    Psi[:] = Psi[:]/big
    # mathematical scaling of Fi for F[rr]=Psi[r]
    coef = Psi[r]/Fi[rr]
    Fi[:] = coef * Fi[:]
    # calculation of f(E) in node of sewing
    curve1 = interp1d(X, Psi, kind='cubic')
    curve2 = interp1d(XX, Fi, kind='cubic')
    der1 = derivative(curve1, X[r], dx=1.e-6)
    der2 = derivative(curve2, XX[rr], dx=1.e-6)
    f = der1-der2
    return f

#this
def average_value(psi, oper_value):
    global X
    value = []
    for ind in range(len(psi)):
        value.append(psi[ind] * oper_value[ind])

    fun = interp1d(X, value, kind='cubic')
    result = quad(fun, A, B)
    return result

#this
def plotting_wf0(e):
    global r, n, Psi, Fi, X, XX
    ff = f_fun(e)
    print("f=", ff)
    plt.axis([A, B, -1, W])
    Upot = np.array([U(X[i]) for i in np.arange(n)])

    coefPsi = average_value(Psi, Psi)
    Psi[:] = Psi[:]/math.sqrt(coefPsi[0])
    normPsi = average_value(Psi, Psi)
    print("integrate Psi = ", normPsi[0])

    coefFi = average_value(Fi, Fi)
    Fi[:] = Fi[:] / math.sqrt(coefFi[0])
    normFi = average_value(Fi, Fi)
    print("integrate Fi = ", normFi[0])

    averageT = average_value(Psi, operT(Psi, X))
    print("<T> = ", averageT[0])

    UPsiValue = []
    for ind in range(len(X)):
        UPsiValue.append(Psi[ind] * U(X[ind]))
    averageU = average_value(Psi, UPsiValue)
    print("<U> = ", averageU[0])

    print("<T> + <U> = ", averageT[0] + averageU[0])

    Psi2 = []
    for psi in Psi:
        Psi2.append(math.pow(psi, 2))

    plt.plot(X, Upot, 'g-', linewidth=6.0, label="U(x)")
    Zero = np.zeros(n, dtype=float)
    plt.plot(X, Zero, 'k-', linewidth=1.0)  # abscissa axis
    plt.plot(X, Psi, 'r-', linewidth=2.0, label="Psi(x)")
    plt.plot(X, Psi2, 'b-', linewidth=2.0, label="Psi^2(x)")
    plt.xlabel("X", fontsize=18, color="k")
    plt.ylabel("Psi(x), Psi^2(x), U(x)", fontsize=18, color="k")
    plt.grid(True)
    plt.legend(fontsize=16, shadow=True, fancybox=True, loc='upper right')
    plt.plot([X[r]], [Psi[r]], color='red', marker='o', markersize=7)
    string1 = "E    = " + format(e, "10.7f")
    string2 = "f(E) = " + format(ff, "10.3e")
    plt.text(-1.5, 2.7, string1, fontsize=14, color='black')
    plt.text(-1.5, 2.3, string2, fontsize=14, color="black")
    # save to file
    name = "glebov-2a.pdf"
    plt.savefig(name, dpi=300)
    plt.show()

#e0 = 0.3913917
#e2 = 1.3083863

# initial data (atomic units)
V0 = 25 / 27.212
L = 2.0 / 0.5292
A = -L
B = +L
# number of mesh node
n = 1001  # odd integer number
print("n=", n)
print("n=", n, file=T)
# minimum of potential (atomic units)
U0 = 0
# maximum of potential (atomic units) - for visualization only!
W = 5.0
# x-coordinates of the nodes
X  = np.linspace(A, B, n)  # forward
XX = np.linspace(B, A, n)  # backwards
# node of sewing
r = (n-1)*3//4      # forward
rr = n-r-1          # backwards
print("r=", r)
print("r=", r, file=T)
print("rr=", rr)
print("rr=", rr, file=T)
print("X[r]=", X[r])
print("X[r]=", X[r], file=T)
print("XX[rr]=", XX[rr])
print("XX[rr]=", XX[rr], file=T)
# input of energy
e = float(input("Energy = "))
print("e =", e)
# plot
dummy = plotting_wf0(e)

In [ ]:
"""
One-dimensional Schrödinger steady-state equation.
Square potential well.
"""
import numpy as np
from scipy.integrate import odeint
from scipy.misc import derivative
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

global r, n, Psi, Fi, X, XX

LST = open("glebov-2b.txt", "wt")


# potential function
def U(x):
    if (abs(x) < L):
        return V0 * pow(x - L, 2) / 9 * np.exp(-x * x)
    else:
        return W


# function (13)
def q(e, x):
    print("e = ", e, ", x = ", x)
    return 2.0*(e-U(x))

def system1(cond1, X):
    global eee
    Y0, Y1 = cond1[0], cond1[1]
    dY0dX = Y1
    dY1dX = - q(eee, X)*Y0
    return [dY0dX, dY1dX]


def system2(cond2, XX):
    global eee
    Z0, Z1 = cond2[0], cond2[1]
    dZ0dX = Z1
    dZ1dX = - q(eee, XX)*Z0
    return [dZ0dX, dZ1dX]


# calculation of f (eq. 18; difference of derivatives)
def f_fun(e):
    global r, n, Psi, Fi, X, XX, eee
    eee = e
    """
    Cauchy problem ("forward")
    dPsi1(x)/dx = - q(e, x)*Psi(x);
    dPsi(x)/dx = Psi1(x);
    Psi(A) = 0.0
    Psi1(A)= 1.0
    """
    cond1 = [0.0, 1.0]
    sol1 = odeint(system1, cond1, X)
    Psi, Psi1 = sol1[:, 0], sol1[:, 1]
    """
    Cauchy problem ("backwards")
    dPsi1(x)/dx = - q(e, x)*Psi(x);
    dPsi(x)/dx = Psi1(x);
    Psi(B) = 0.0
    Psi1(B)= 1.0
    """
    cond2 = [0.0, 1.0]
    sol2 = odeint(system2, cond2, XX)
    Fi, Fi1 = sol2[:, 0], sol2[:, 1]
    # search of maximum value of Psi
    p1 = np.abs(Psi).max()
    p2 = np.abs(Psi).min()
    big = p1 if p1 > p2 else p2
    # scaling of Psi
    Psi[:] = Psi[:]/big
    # mathematical scaling of Fi for F[rr]=Psi[r]
    coef = Psi[r]/Fi[rr]
    Fi[:] = coef * Fi[:]
    # calculation of f(E) in node of sewing
    curve1 = interp1d(X, Psi, kind='cubic')
    curve2 = interp1d(XX, Fi, kind='cubic')
    der1 = derivative(curve1, X[r], dx=1.e-6)
    der2 = derivative(curve2, XX[rr], dx=1.e-6)
    f = der1-der2
    return f


def m_bis(x1, x2, tol):
    global r, n
    if f_fun(e=x2)*f_fun(e=x1) > 0.0:
        print("ERROR: f_fun(e=x2, r, n)*f_fun(e=x1, r, n) > 0")
        print("x1=", x1)
        print("x2=", x2)
        print("f_fun(e=x1, r=r, n=n)=", f_fun(e=x1))
        print("f_fun(e=x2, r=r, n=n)=", f_fun(e=x2))
        exit()
    while abs(x2-x1) > tol:
        xr = (x1+x2)/2.0
        if f_fun(e=x2)*f_fun(e=xr) < 0.0:
            x1 = xr
        else:
            x2 = xr
        if f_fun(e=x1)*f_fun(e=xr) < 0.0:
            x2 = xr
        else:
            x1 = xr
    return (x1+x2)/2.0


def plotting_f():
    plt.axis([U0, e2, fmin, fmax])
    ZeroE = np.zeros(ne, dtype=float)
    plt.plot(ee, ZeroE, 'k-', linewidth=1.0)  # abscissa axis
    plt.plot(ee, af, 'bo', markersize=2)
    plt.xlabel("E", fontsize=18, color="k")
    plt.ylabel("f(E)", fontsize=18, color="k")
    plt.grid(True)
    # save to file
    plt.savefig('glebov-2b-f.pdf', dpi=300)
    plt.show()


def plotting_wf(e):
    global r, n, Psi, Fi, X, XX
    ff = f_fun(e)
    plt.axis([A, B, -1, W])
    Upot = np.array([U(X[i]) for i in np.arange(n)])
    plt.plot(X, Upot, 'g-', linewidth=6.0, label="U(x)")
    Zero = np.zeros(n, dtype=float)
    plt.plot(X, Zero, 'k-', linewidth=1.0)  # abscissa axis
    plt.plot(X, Psi, 'r-', linewidth=2.0, label="Psi(x)")
    plt.plot(XX, Fi, 'b-', linewidth=2.0, label="Fi(x)")
    plt.xlabel("X", fontsize=18, color="k")
    plt.ylabel("Psi(x), Fi(x), U(x)", fontsize=18, color="k")
    plt.grid(True)
    plt.legend(fontsize=16, shadow=True, fancybox=True, loc='upper right')
    plt.plot([X[r]], [Psi[r]], color='red', marker='o', markersize=7)
    string1 = "E    = " + format(e, "10.7f")
    string2 = "f(E) = " + format(ff, "10.3e")
    plt.text(-1.5, 2.7, string1, fontsize=14, color='black')
    plt.text(-1.5, 2.3, string2, fontsize=14, color="black")
    # save to file
    name = "glebov-2b" + "-" + str(ngr) + ".pdf"
    plt.savefig(name, dpi=300)
    #plt.show()

# initial data (atomic units)
V0 = 25 / 27.212
L = 2.0 / 0.5292
A = -L
B = +L
# number of mesh node
n = 1001  # odd integer number
print("n=", n)
print("n=", n, file=LST)
# minimum of potential (atomic units)
U0 = 0
# maximum of potential (atomic units) - for visualization only!
W = 5.0
# x-coordinates of the nodes
X  = np.linspace(A, B, n)  # forward
XX = np.linspace(B, A, n)  # backwards
# node of sewing
r = (n-1)*3//4      # forward
rr = n-r-1          # backwards
print("r=", r)
print("r=", r, file=LST)
print("rr=", rr)
print("rr=", rr, file=LST)
print("X[r]=", X[r])
print("X[r]=", X[r], file=LST)
print("XX[rr]=", XX[rr])
print("XX[rr]=", XX[rr], file=LST)
# plot of f(e)
e1 = U0+0.05
e2 = 10.0
print("e1=", e1, "   e2=", e2)
print("e1=", e1, "   e2=", e2, file=LST)
ne = 101
print("ne=", ne)
print("ne=", ne, file=LST)
ee = np.linspace(e1, e2, ne)
af = np.zeros(ne, dtype=float)
porog = 5.0
tol = 1.0e-7
energy = []
ngr = 0
for i in np.arange(ne):
    e = ee[i]
    af[i] = f_fun(e)
    stroka = "i = {:3d}   e = {:8.5f}  f[e] = {:12.5e}"
    print(stroka.format(i, e, af[i]))
    print(stroka.format(i, e, af[i]), file=LST)
    if i > 0:
        Log1 = af[i]*af[i-1] < 0.0
        Log2 = np.abs(af[i]-af[i-1]) < porog
        if Log1 and Log2:
            energy1 = ee[i-1]
            energy2 = ee[i]
            eval = m_bis(energy1, energy2, tol)
            print("eval = {:12.5e}".format(eval))
            dummy = plotting_wf(eval)
            energy.append(eval)
            ngr += 1

fmax = +10.0
fmin = -10.0
# plot
dummy = plotting_f()
# output of roots
nroots = len(energy)
print("nroots =", nroots)
print("nroots =", nroots, file=LST)
for i in np.arange(nroots):
    stroka = "i = {:1d}    energy[i] = {:12.5e}"
    print(stroka.format(i, energy[i]))
    print(stroka.format(i, energy[i]), file=LST)


n= 1001
r= 750
rr= 250
X[r]= 1.88964474679
XX[rr]= 1.88964474679
e1= 0.05    e2= 10.0
ne= 101
e =  0.05 , x =  -3.7792894935752077
e =  0.05 , x =  -3.7791714843219215
e =  0.05 , x =  -3.7791714843219215
e =  0.05 , x =  -3.7790534750686353
e =  0.05 , x =  -3.7790534750686353
e =  0.05 , x =  -3.7703840729046814
e =  0.05 , x =  -3.7703840729046814
e =  0.05 , x =  -3.7617146707407274
e =  0.05 , x =  -3.7617146707407274
e =  0.05 , x =  -3.7530452685767735
e =  0.05 , x =  -3.7530452685767735
e =  0.05 , x =  -3.689279152241391
e =  0.05 , x =  -3.689279152241391
e =  0.05 , x =  -3.717137057343021
e =  0.05 , x =  -3.717137057343021
e =  0.05 , x =  -3.745863626330023
e =  0.05 , x =  -3.745863626330023
e =  0.05 , x =  -3.7386819840832723
e =  0.05 , x =  -3.7386819840832723
e =  0.05 , x =  -3.731500341836522
e =  0.05 , x =  -3.731500341836522
e =  0.05 , x =  -3.7243186995897712
e =  0.05 , x =  -3.7243186995897712
e =  0.05 , x =  -3.70995541509627
e =  0.05 , x =  -3.70995541

e =  0.05 , x =  2.6092455149940563
e =  0.05 , x =  2.6989523065135312
e =  0.05 , x =  2.6989523065135312
e =  0.05 , x =  2.788659098033006
e =  0.05 , x =  2.788659098033006
e =  0.05 , x =  2.878365889552481
e =  0.05 , x =  2.878365889552481
e =  0.05 , x =  2.968072681071956
e =  0.05 , x =  2.968072681071956
e =  0.05 , x =  3.057779472591431
e =  0.05 , x =  3.057779472591431
e =  0.05 , x =  3.147486264110906
e =  0.05 , x =  3.147486264110906
e =  0.05 , x =  3.2371930556303807
e =  0.05 , x =  3.2371930556303807
e =  0.05 , x =  3.3978124075313376
e =  0.05 , x =  3.3978124075313376
e =  0.05 , x =  3.5584317594322945
e =  0.05 , x =  3.5584317594322945
e =  0.05 , x =  3.7190511113332514
e =  0.05 , x =  3.7190511113332514
e =  0.05 , x =  3.685717195808909
e =  0.05 , x =  3.685717195808909
e =  0.05 , x =  3.8130026321855235
e =  0.05 , x =  3.8130026321855235
e =  0.05 , x =  3.8130026321855235
e =  0.05 , x =  3.7175385549030624
e =  0.05 , x =  3.7175385549030624
e = 

e =  0.05 , x =  1.0695567125343752
e =  0.05 , x =  1.0695567125343752
e =  0.05 , x =  1.00118633377176
e =  0.05 , x =  1.00118633377176
e =  0.05 , x =  0.9328159550091446
e =  0.05 , x =  0.9328159550091446
e =  0.05 , x =  0.8644455762465293
e =  0.05 , x =  0.8644455762465293
e =  0.05 , x =  0.796075197483914
e =  0.05 , x =  0.796075197483914
e =  0.05 , x =  0.7277048187212987
e =  0.05 , x =  0.7277048187212987
e =  0.05 , x =  0.6593344399586833
e =  0.05 , x =  0.6593344399586833
e =  0.05 , x =  0.590964061196068
e =  0.05 , x =  0.590964061196068
e =  0.05 , x =  0.5225936824334527
e =  0.05 , x =  0.5225936824334527
e =  0.05 , x =  0.4542233036708374
e =  0.05 , x =  0.4542233036708374
e =  0.05 , x =  0.467643387413309
e =  0.05 , x =  0.467643387413309
e =  0.05 , x =  0.4126930923931653
e =  0.05 , x =  0.4126930923931653
e =  0.05 , x =  0.3577427973730216
e =  0.05 , x =  0.3577427973730216
e =  0.05 , x =  0.3027925023528779
e =  0.05 , x =  0.3027925023528779
e 

e =  0.1495 , x =  -3.6965854302697805
e =  0.1495 , x =  -3.6110098429381248
e =  0.1495 , x =  -3.6110098429381248
e =  0.1495 , x =  -3.525434255606469
e =  0.1495 , x =  -3.525434255606469
e =  0.1495 , x =  -3.4398586682748133
e =  0.1495 , x =  -3.4398586682748133
e =  0.1495 , x =  -3.3542830809431576
e =  0.1495 , x =  -3.3542830809431576
e =  0.1495 , x =  -3.268707493611502
e =  0.1495 , x =  -3.268707493611502
e =  0.1495 , x =  -3.183131906279846
e =  0.1495 , x =  -3.183131906279846
e =  0.1495 , x =  -3.0975563189481905
e =  0.1495 , x =  -3.0975563189481905
e =  0.1495 , x =  -3.011980731616535
e =  0.1495 , x =  -3.011980731616535
e =  0.1495 , x =  -2.926405144284879
e =  0.1495 , x =  -2.926405144284879
e =  0.1495 , x =  -2.8408295569532234
e =  0.1495 , x =  -2.8408295569532234
e =  0.1495 , x =  -2.8565080205305375
e =  0.1495 , x =  -2.8565080205305375
e =  0.1495 , x =  -2.786610896776196
e =  0.1495 , x =  -2.786610896776196
e =  0.1495 , x =  -2.716713773021854

e =  0.1495 , x =  3.7793261491402834
e =  0.1495 , x =  3.7793261491402834
e =  0.1495 , x =  3.7793261491402834
e =  0.1495 , x =  3.7793261491402834
e =  0.1495 , x =  3.7792851802122023
e =  0.1495 , x =  3.7792954224442226
e =  0.1495 , x =  3.7792954224442226
e =  0.1495 , x =  3.7792954224442226
e =  0.1495 , x =  3.7792954224442226
e =  0.1495 , x =  3.7792852459019044
e =  0.1495 , x =  3.7792853115916065
e =  0.1495 , x =  3.7792854429710103
e =  0.1495 , x =  3.779285574350414
e =  0.1495 , x =  3.779285705729818
e =  0.1495 , x =  3.779287019523857
e =  0.1495 , x =  3.7792883333178957
e =  0.1495 , x =  3.7792896471119346
e =  0.1495 , x =  3.7792896471119346
e =  0.1495 , x =  3.7792896471119346
e =  0.1495 , x =  3.7792896471119346
e =  0.1495 , x =  3.7792883568447793
e =  0.1495 , x =  3.779288380371663
e =  0.1495 , x =  3.77928842742543
e =  0.1495 , x =  3.779288474479197
e =  0.1495 , x =  3.779288521532964
e =  0.1495 , x =  3.7792889920706356
e =  0.1495 , x =  3

...

e =  0.391399658203 , x =  1.5282560023170992
e =  0.391399658203 , x =  1.476567810593201
e =  0.391399658203 , x =  1.476567810593201
e =  0.391399658203 , x =  1.4248796188693027
e =  0.391399658203 , x =  1.4248796188693027
e =  0.391399658203 , x =  1.3731914271454044
e =  0.391399658203 , x =  1.3731914271454044
e =  0.391399658203 , x =  1.3074629894505039
e =  0.391399658203 , x =  1.3074629894505039
e =  0.391399658203 , x =  1.2417345517556035
e =  0.391399658203 , x =  1.2417345517556035
e =  0.391399658203 , x =  1.1760061140607032
e =  0.391399658203 , x =  1.1760061140607032
e =  0.391399658203 , x =  1.189370548935678
e =  0.391399658203 , x =  1.189370548935678
e =  0.391399658203 , x =  1.1370065461157526
e =  0.391399658203 , x =  1.1370065461157526
e =  0.391399658203 , x =  1.084642543295827
e =  0.391399658203 , x =  1.084642543295827
e =  0.391399658203 , x =  1.0322785404759016
e =  0.391399658203 , x =  1.0322785404759016
e =  0.391399658203 , x =  0.97991453765

e =  0.391387512207 , x =  -2.773772716146378
e =  0.391387512207 , x =  -2.701861112689257
e =  0.391387512207 , x =  -2.701861112689257
e =  0.391387512207 , x =  -2.6299495092321363
e =  0.391387512207 , x =  -2.6299495092321363
e =  0.391387512207 , x =  -2.5580379057750156
e =  0.391387512207 , x =  -2.5580379057750156
e =  0.391387512207 , x =  -2.4861263023178948
e =  0.391387512207 , x =  -2.4861263023178948
e =  0.391387512207 , x =  -2.401552814081311
e =  0.391387512207 , x =  -2.401552814081311
e =  0.391387512207 , x =  -2.316979325844727
e =  0.391387512207 , x =  -2.316979325844727
e =  0.391387512207 , x =  -2.2324058376081433
e =  0.391387512207 , x =  -2.2324058376081433
e =  0.391387512207 , x =  -2.252423979999452
e =  0.391387512207 , x =  -2.252423979999452
e =  0.391387512207 , x =  -2.187868634154177
e =  0.391387512207 , x =  -2.187868634154177
e =  0.391387512207 , x =  -2.123313288308902
e =  0.391387512207 , x =  -2.123313288308902
e =  0.391387512207 , x = 

e =  0.391387512207 , x =  3.547046897407462
e =  0.391387512207 , x =  3.488462468561421
e =  0.391387512207 , x =  3.488462468561421
e =  0.391387512207 , x =  3.42987803971538
e =  0.391387512207 , x =  3.42987803971538
e =  0.391387512207 , x =  3.3712936108693388
e =  0.391387512207 , x =  3.3712936108693388
e =  0.391387512207 , x =  3.2495610110257536
e =  0.391387512207 , x =  3.2495610110257536
e =  0.391387512207 , x =  3.1278284111821684
e =  0.391387512207 , x =  3.1278284111821684
e =  0.391387512207 , x =  3.006095811338583
e =  0.391387512207 , x =  3.006095811338583
e =  0.391387512207 , x =  2.884363211494998
e =  0.391387512207 , x =  2.884363211494998
e =  0.391387512207 , x =  2.7626306116514128
e =  0.391387512207 , x =  2.7626306116514128
e =  0.391387512207 , x =  2.6408980118078276
e =  0.391387512207 , x =  2.6408980118078276
e =  0.391387512207 , x =  2.5191654119642424
e =  0.391387512207 , x =  2.5191654119642424
e =  0.391387512207 , x =  2.5382164345719262

e =  0.391387512207 , x =  -3.7791714843219215
e =  0.391387512207 , x =  -3.7791714843219215
e =  0.391387512207 , x =  -3.7790534750686353
e =  0.391387512207 , x =  -3.7790534750686353
e =  0.391387512207 , x =  -3.774687079460955
e =  0.391387512207 , x =  -3.774687079460955
e =  0.391387512207 , x =  -3.7703206838532743
e =  0.391387512207 , x =  -3.7703206838532743
e =  0.391387512207 , x =  -3.765954288245594
e =  0.391387512207 , x =  -3.765954288245594
e =  0.391387512207 , x =  -3.742487373527277
e =  0.391387512207 , x =  -3.742487373527277
e =  0.391387512207 , x =  -3.7505457250149012
e =  0.391387512207 , x =  -3.7505457250149012
e =  0.391387512207 , x =  -3.7628725755994554
e =  0.391387512207 , x =  -3.7628725755994554
e =  0.391387512207 , x =  -3.759790862953317
e =  0.391387512207 , x =  -3.759790862953317
e =  0.391387512207 , x =  -3.7567091503071786
e =  0.391387512207 , x =  -3.7567091503071786
e =  0.391387512207 , x =  -3.75362743766104
e =  0.391387512207 , x

e =  0.391387512207 , x =  3.1808162365223036
e =  0.391387512207 , x =  3.278652214690526
e =  0.391387512207 , x =  3.278652214690526
e =  0.391387512207 , x =  3.3764881928587482
e =  0.391387512207 , x =  3.3764881928587482
e =  0.391387512207 , x =  3.4743241710269706
e =  0.391387512207 , x =  3.4743241710269706
e =  0.391387512207 , x =  3.572160149195193
e =  0.391387512207 , x =  3.572160149195193
e =  0.391387512207 , x =  3.669996127363415
e =  0.391387512207 , x =  3.669996127363415
e =  0.391387512207 , x =  3.7451931852324414
e =  0.391387512207 , x =  3.7451931852324414
e =  0.391387512207 , x =  3.8203902431014676
e =  0.391387512207 , x =  3.8203902431014676
e =  0.391387512207 , x =  3.8203902431014676
e =  0.391387512207 , x =  3.763992449699698
e =  0.391387512207 , x =  3.763992449699698
e =  0.391387512207 , x =  3.7827917141669545
e =  0.391387512207 , x =  3.7827917141669545
e =  0.391387512207 , x =  3.7827917141669545
e =  0.391387512207 , x =  3.7827917141669